## Data Exploration, Cleanup and Analysis

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [77]:
#Essentials
import numpy as np
import pandas as pd

#SQL related
import sqlite3
import pandas.io.sql as pd_sql

#Plotting fun
import matplotlib.pyplot as plt

#Nice to have
import seaborn as sns
import re
from calendar import month_name

#NLP modules
from geotext import GeoText
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from iata_codes import IATACodesClient

%matplotlib inline

In [3]:
#Setting up for working with SQLite database
sqlite_file = '/Users/auste_m/ds/metis/metisgh/github/metis_projects/Customer_Review_Sentiment_Analysis/Datasets/twitter-airline-sentiment/database.sqlite'

conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()

In [4]:
#Check one of the rows in the table
preview = cursor.execute("SELECT * FROM Tweets LIMIT 20")
columns = [column[0] for column in preview.description]
print('The columns of the table are:' + ' \n' + str(columns) + '\n')
print('Preview of one of the rows in the table:' + '\n' + str(preview.fetchone()))

The columns of the table are: 
['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence', 'airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone']

Preview of one of the rows in the table:
(567588278875213824, 'neutral', 1, '', '', 'Delta', '', 'JetBlueNews', '', 0, "@JetBlue's new CEO seeks the right balance to please passengers and Wall ... - Greenfield Daily Reporter http://t.co/LM3opxkxch", '', '2015-02-16 23:36:05 -0800', 'USA', 'Sydney')


In [5]:
#Retrieve relevant information from Tweets table in SQLite database and store them in a pandas dataframe
query = """SELECT airline, retweet_count, text as 'tweet' 
            FROM Tweets"""


tweets_df = pd.read_sql_query(query, conn)

In [6]:
#Sanity check
print(tweets_df.info())
tweets_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14485 entries, 0 to 14484
Data columns (total 3 columns):
airline          14485 non-null object
retweet_count    14485 non-null int64
tweet            14485 non-null object
dtypes: int64(1), object(2)
memory usage: 339.6+ KB
None


,airline,retweet_count,tweet
0,Delta,0,@JetBlue's new CEO seeks the right balance to ...
1,Delta,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...
2,United,0,@united yes. We waited in line for almost an h...
3,United,0,@united the we got into the gate at IAH on tim...
4,Southwest,0,@SouthwestAir its cool that my bags take a bit...


### Let's have some regex fun!

In [71]:
#helper function to remove stuff from tweets

def airline_remove(string):
    """Takes a string as input.
    Returns the same string with hashtag removed."""
    pattern1 = re.compile('@[A-Za-z]+\w')
    new_string = string
    try:
        all_airlines = pattern1.findall(new_string)
        for airline in all_airlines:
            new_string = re.sub(airline, '', new_string)
    except:
        pass
    return new_string

def hashtag_remove(string):
    """Takes a string as input.
    Returns the same string with hashtag removed."""
    pattern2 = re.compile('#\w+')
    new_string = string
    try:
        all_hashtags = pattern2.findall(new_string)
        for hashtag in all_hashtags:
            new_string = re.sub(hashtag, '', new_string)
    except:
        pass
    return new_string


def code_remove(string):
    """Takes a string as input.
    Returns the same string with any capital letter & digit combination text removed."""
    pattern3 = re.compile('[A-Z]?\d+[A-Z]+')
    pattern4 = re.compile('\d+')
    new_string = string
    try:
        to_be_removed = pattern3.findall(new_string)
        to_be_removed.extend(pattern4.findall(new_string))
        for elem in to_be_removed:
            new_string = re.sub(elem, '', new_string)
    except:
        pass
    return new_string
    
    
def url_remove(string):
    """Takes a string as input.
    Returns the same string with any urls removed removed."""
    pattern5 = re.compile('http://t.co/\w+')
    new_string = string
    try:
        urls = pattern5.findall(new_string)
        for url in urls:
            new_string = re.sub(url, '', new_string)
    except:
        pass
    return new_string


def location_remove(string):
    """Takes a string as input.
    Returns a new string with location information removed."""
    new_string = string
    geo_loc = GeoText(string)
    locations = []
    if geo_loc.cities != []:
        locations.extend(geo_loc.cities)
    if geo_loc.countries != []:
        locations.extend(geo_loc.countries)
    try:
        for loc in locations:
            new_string = re.sub(loc, '', new_string)
    except:
        pass
    return new_string
    
    
def month_remove(string):
    """Takes a string as input.
    Returns a new string with month information removed."""
    new_string = string
    all_months = month_name[1:]
    try:
        for month in all_months:
            new_string = re.sub(month, '', new_string)
    except:
        pass
    return new_string


def emoji_remove(string):
    """Takes a string as input.
    Returns a new string with emojis removed."""    
    pattern6 = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    new_string = string
    try:
        emojis = pattern6.findall(new_string)
        for emoji in emojis:
            new_string = re.sub(emoji, '', new_string)
    except:
        pass
    return new_string
    

In [72]:
#Clean up tweet column, remove the "@word" from the rest of the tweet
tweets_df['tweet_clean'] = tweets_df['tweet'].apply(airline_remove)

#Clean up tweet column, remove the hashtags from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(hashtag_remove)

#Clean up tweet column, remove code-like elements from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(code_remove)

#Clean up tweet column, remove urls from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(url_remove)

#Clean up tweet column, remove urls from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(location_remove)

#Clean up tweet column, remove month names from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(month_remove)

#Clean up tweet column, remove month names from all tweets
tweets_df['tweet_clean'] = tweets_df['tweet_clean'].apply(emoji_remove)

#### Examining hashtag containing tweets

In [9]:
# #Identify all the tweets containing hashtags
# pattern2 = re.compile('#[A-Za-z]+\w')
# count_hash_tweets = 0

# for index, tweet in enumerate(tweets_df['tweet']):
#     try:
#         h_tweet = pattern2.search(tweet).group()
#         print(index, h_tweet)
#         count_hash_tweets += 1
#     except:
#         continue
        
# print('\nTotal number of tweets containing hashtags =', str(count_hash_tweets))

#### Same for urls

In [10]:
# # Find url pattern
# https_list = []

# for tweet in tweets_df['tweet']:
#     if url_remove(tweet) == []:
#         pass
#     else:
#         https_list.append(url_remove(tweet))
        
# print(https_list)

In [11]:
# #Testing code_remove function
# test_string = tweets_df['tweet'][1582]
# print(test_string)
# print(url_remove(test_string))

> Wooohooo!!!

In [ ]:
#Clean up tweet column, remove urls from all tweets
tweets_df['tweet_no_urls'] = tweets_df['tweet_no_airline'].apply(hashtag_remove)

#### Now it's location time

In [74]:
# madrid_tweet = tweets_df['tweet'][14451]
# geo = GeoText(madrid_tweet)
# geo.cities

#### Let's try to remove the airport codes

#### Mini moment of truth

In [73]:
#Let's make sure it works
print(tweets_df['tweet'][1], '\n')
print(tweets_df['tweet_clean'][1], '\n')

@JetBlue is REALLY getting on my nerves !! 😡😡 #nothappy 

 is REALLY getting on my nerves !!   



In [76]:
tweets_df.head(10)

,airline,retweet_count,tweet,tweet_clean
0,Delta,0,@JetBlue's new CEO seeks the right balance to ...,'s new CEO seeks the right balance to please p...
1,Delta,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,is REALLY getting on my nerves !!
2,United,0,@united yes. We waited in line for almost an h...,yes. We waited in line for almost an hour to ...
3,United,0,@united the we got into the gate at IAH on tim...,the we got into the gate at IAH on time and h...
4,Southwest,0,@SouthwestAir its cool that my bags take a bit...,"its cool that my bags take a bit longer, dont..."
5,United,0,@united and don't hope for me having a nicer f...,and don't hope for me having a nicer flight s...
6,United,0,@united I like delays less than you because I'...,I like delays less than you because I'm the o...
7,United,0,"@united, link to current status of flights/air...",", link to current status of flights/airports? ..."
8,Southwest,0,@SouthwestAir you guys there? Are we on hour 2...,you guys there? Are we on hour of our phone ...
9,United,0,@united I tried 2 DM it would not go thru... n...,I tried DM it would not go thru... not sure why


In [ ]:
# class RecommendationEngine:
    
#     def __init__(self, vectorizer, n_components, reducer):
#         self.vectorizer = vectorizer
#         self.n_dim = n_components
#         self.reducer = reducer(n_components)
        
#     def fit(self, text):
#         self.vector_data = self.vectorizer.fit_transform(text)
#         self.topic_data = self.reducer.fit_transform(self.vector_data)
#         self.text = text
        
#     def recommend(self, article, num_to_return):